First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

ModuleNotFoundError: No module named 'matplotlib'

Initalize variables

In [52]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [53]:
data = pd.read_csv(f'{path}/data/btw_treatment.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
data.head(10)

,AGS,GEN,year,Land,post_2005,post_2009,post_2013,post_2017,Erststimme,Zweitstimme,...,avg. income,avg. age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,protestant,catholic,religion other/none
0,1001000,Flensburg,2002.0,SH,0.0,0.0,0.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
1,1001000,Flensburg,2002.0,SH,0.0,0.0,0.0,0.0,NaN,1.0,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
2,1001000,Flensburg,2005.0,SH,1.0,0.0,0.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
3,1001000,Flensburg,2005.0,SH,1.0,0.0,0.0,0.0,NaN,1.0,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
4,1001000,Flensburg,2009.0,SH,1.0,1.0,0.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
5,1001000,Flensburg,2009.0,SH,1.0,1.0,0.0,0.0,NaN,1.0,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
6,1001000,Flensburg,2013.0,SH,1.0,1.0,1.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
7,1001000,Flensburg,2013.0,SH,1.0,1.0,1.0,0.0,NaN,1.0,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
8,1001000,Flensburg,2017.0,SH,1.0,1.0,1.0,1.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
9,1001000,Flensburg,2017.0,SH,1.0,1.0,1.0,1.0,NaN,1.0,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726


Split data in Erst- and Zweitstimme

In [54]:
erst = data[data['Erststimme'] == 1]
zweit = data[data['Zweitstimme'] == 1]
erst.head(10)

,AGS,GEN,year,Land,post_2005,post_2009,post_2013,post_2017,Erststimme,Zweitstimme,...,avg. income,avg. age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,protestant,catholic,religion other/none
0,1001000,Flensburg,2002.0,SH,0.0,0.0,0.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
2,1001000,Flensburg,2005.0,SH,1.0,0.0,0.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
4,1001000,Flensburg,2009.0,SH,1.0,1.0,0.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
6,1001000,Flensburg,2013.0,SH,1.0,1.0,1.0,0.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
8,1001000,Flensburg,2017.0,SH,1.0,1.0,1.0,1.0,1.0,NaN,...,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
10,1002000,Kiel,2002.0,SH,0.0,0.0,0.0,0.0,1.0,NaN,...,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436
12,1002000,Kiel,2005.0,SH,1.0,0.0,0.0,0.0,1.0,NaN,...,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436
14,1002000,Kiel,2009.0,SH,1.0,1.0,0.0,0.0,1.0,NaN,...,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436
16,1002000,Kiel,2013.0,SH,1.0,1.0,1.0,0.0,1.0,NaN,...,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436
18,1002000,Kiel,2017.0,SH,1.0,1.0,1.0,1.0,1.0,NaN,...,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436


Statsmodels

DiD for primary vote, treatment 50 in 2013

In [55]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2013 + Land', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])}) #locals converts string to variable name
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/erst_2013.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for primary vote, treatment 50 in 2017

In [56]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2017 + Land', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2017')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/erst_2017.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for Secondary vote, treatment 50 in 2013

In [57]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2013  + Land', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/zweit_2013.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for seconday vote, treatment 50 in 2017

In [58]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2017 + Land', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) #np.array(zweit[['AGS', 'year']])
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2017')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/zweit_2017.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Placebo DiD for treatments in 2005 and 2009

Erststimme post 2005

In [59]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2005 + Land', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2005')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/erst_2005.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Erststimme post 2009

In [60]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2009 + Land', data=erst).fit(cov_type='cluster', cov_kwds={'groups': np.array(erst[['AGS', 'year']])})
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2009')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/erst_2009.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Zweitstimme post 2005

In [61]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2005 + Land', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) # zweit['AGS']
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2005')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/zweit_2005.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

Zweitstimme post 2009

In [62]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2009 + Land', data=zweit).fit(cov_type='cluster', cov_kwds={'groups': np.array(zweit[['AGS', 'year']])}) # zweit['AGS']
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Secondary Vote, Municipalities within 50 km of a powerline project, treatment year 2009')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/btw/zweit_2009.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())